<a href="https://colab.research.google.com/github/perfectpanda-works/machine-learning/blob/master/PyTorchTutorial_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#PyTorchの畳み込みニューラルネットワークのチュートリアル

#Pythonのクラスの復習

In [ ]:
# class test
class parent_class:
  #コンストラクタ
  def __init__(self):
    print("Initialize_parent")

  def func1(self,num):
    print("parent")
    print(num)

class child_class(parent_class):
  #コンストラクタ
  #インスタンス生成時に一度だけ呼び出される
  def __init__(self):
    super().__init__()
    print("Initialize_child")

  def fnc2(self,num):
    super().func1(num)
    print("child")
    print(num)

In [ ]:
#child_classのインスタンスを作成
t_instance = child_class()
t_instance2 = child_class()

Initialize_parent
Initialize_child
Initialize_parent
Initialize_child


In [ ]:
#メソッドにアクセス
t_instance.fnc2(1)

parent
1
child
1


#畳み込みニューラルネットワーク
PyTorchでニューラルネットワークを作成するために必要なimportです。


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

クラスでニューラルネットワークを定義していきます。

In [ ]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [ ]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


#パラメータの確認

In [ ]:
params = list(net.parameters())
print(len(params))
#print(params[0].size())  # conv1's .weight
for i in range(0,10):
  print(params[i].size())

10
torch.Size([6, 1, 3, 3])
torch.Size([6])
torch.Size([16, 6, 3, 3])
torch.Size([16])
torch.Size([120, 576])
torch.Size([120])
torch.Size([84, 120])
torch.Size([84])
torch.Size([10, 84])
torch.Size([10])


次のようにして、Conv1層のパラメータの中身を確認してみます。
Conv1層のパラメータとして３×３のフィルターが６個あるのがわかります。

In [ ]:
print(params[0])
print(params[1])

Parameter containing:
tensor([[[[-0.0933,  0.2394, -0.0573],
          [-0.3326,  0.2173, -0.2807],
          [-0.2363, -0.1357, -0.3250]]],


        [[[ 0.0865,  0.2505, -0.0520],
          [-0.3266,  0.0914,  0.0318],
          [-0.0620, -0.0419, -0.1124]]],


        [[[-0.0704,  0.0988,  0.0958],
          [ 0.1062,  0.2237, -0.1996],
          [-0.2870,  0.1163,  0.2503]]],


        [[[ 0.2017,  0.2823,  0.3023],
          [-0.2970, -0.1573,  0.0235],
          [ 0.3097, -0.1511,  0.0654]]],


        [[[-0.2015, -0.1036, -0.1227],
          [-0.2294, -0.2890,  0.2507],
          [-0.2039, -0.1032, -0.0261]]],


        [[[ 0.2552,  0.2075,  0.2799],
          [ 0.0709, -0.2208,  0.2277],
          [-0.1452, -0.1961,  0.1781]]]], requires_grad=True)
Parameter containing:
tensor([ 0.0031,  0.3142, -0.2670,  0.2170, -0.2200,  0.2434],
       requires_grad=True)


不明点：

①畳み込み１で28×28になる計算（どのように調べて計算しても30×30になってしまう）

②畳み込み２層目、入力が６チャンネルで、出力が１６チャンネルだと、それぞれチャンネルが均等な枚数のフィルターではない？

解決：

どうやら画像とチュートリアルの内容は違うようで、畳み込み層１は30×30のサイズの画像が出力される模様。昔は5×5のフィルターだった？

#画像を入力してみる
ランダムな32×32の行列を入力してみる（画像を見立てた入力）

入力のデータ形式は、バッチサイズ、チャンネル数、画像縦サイズ、画像横サイズという形式の４階のテンソルで表現

通常のカラー画像を入力するさいは、バッチサイズ（まとまった枚数）の画像をRGBという３チャンネルで入力するため、このようなテンソルを入力としている。

今回の入力データは、32×32の1チャンネル（グレースケール、白黒データのみの１チャンネル画像）1枚のみなので、バッチサイズ１の1チャンネル、32×32という形のテンソルを作成する。

In [ ]:
input = torch.randn(1, 1, 32, 32)
#(バッチサイズ,チャンネル,画像縦,画像横)
print(input)

tensor([[[[-1.1831,  0.5253,  0.0705,  ..., -0.5904, -0.1145, -0.7488],
          [-0.9510,  0.2303, -0.6416,  ...,  0.7960,  1.6276,  0.3620],
          [-0.1249,  0.4901, -0.0790,  ..., -0.8723, -0.2881,  0.4356],
          ...,
          [ 0.6352,  0.3252, -0.0446,  ..., -1.8833, -0.0868, -0.2000],
          [ 0.9172,  0.4551,  0.8795,  ...,  0.1327, -0.9136, -0.0452],
          [-0.2299,  0.1815, -1.2079,  ..., -1.3954,  1.2479,  1.0279]]]])


画像をネットワークに入力。

outに10個の値が出力される。10分類問題の場合、これが一番大きい数値がネットワークが予測する答えとなる。

In [ ]:
out = net(input)
print(out)

tensor([[ 0.0197,  0.0280, -0.0223, -0.0835, -0.0448,  0.1229, -0.0594, -0.1072,
          0.0999, -0.0607]], grad_fn=<AddmmBackward>)


#勾配を求める

ニューラルネットワークの勾配はゼロで初期化する。


出力が1×10の形なので、backwordには次のようなテンソル(ベクトル)を指定する。今回は、randnで生成したランダムな勾配(10個の値が入ったベクトル)を逆伝播する。

In [ ]:
#勾配の初期化
net.zero_grad()
#backwordメソッドで勾配を求める
out.backward(torch.randn(1, 10))

#損失関数
損失関数は、(出力,正解)という引数を与えて、モデルから出力された回答がどれだけ正解から遠いかを表現してくれるものです。


PyTorchのnnパッケージには、いくつかの損失関数が実装されていますが、チュートリアルでは、一番シンプルな「nn.MSELoss」を利用します。


これは、平均二乗誤差などと呼ばれる誤差関数になります。

In [ ]:
output = net(input)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.6772, grad_fn=<MseLossBackward>)


outputには、randnで作成したランダムな出力を与えます。


target変数には、ランダムな正解を与えます。


nn.MSELossとしてcriterion変数に誤差関数をインスタンス化します。

criterion(output,target)とすることで、モデルの出力した値と正解を誤差関数で比較します。

#バックプロパゲーション
先の勾配を求めるというところで行なった操作と同じになります。勾配の初期化→勾配を求める、という流れになります。

誤差関数で求めた誤差に対して、微分操作(backword)を行うことで、ニューラルネットワークの各パラメータの勾配を求めることができます。

In [ ]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0014,  0.0065,  0.0212, -0.0037, -0.0007, -0.0094])


バックプロパゲーションを行う前に、勾配を初期化します。

チュートリアルでは、畳み込み層１のバイアスパラメータ（６個の重みがある）がどのように変化するか、確認をしています。

#重みの更新を行う
ニューラルネットワークのもっともシンプルな重み（パラメータ）の更新方法は「確率的勾配降下法(SDG)」という方法で行われます。

・確率的勾配降下法

更新後の重み　= 現在の重み - 学習率　×　勾配

このような式で一回の訓練（１エポック終了後）に重みを更新することになります。

先ほどのbackwordの流れですでに勾配を求めていますので、あとは学習率と掛け合わせて現在の重みから引き算し、その結果を新しい重みとして反映するだけです。

SDGの実装例としては次のようなプログラムになります。


In [ ]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

学習率は、手動で設定する「ハイパーパラメータ」です。この値が大きいほど、早く重みが最適な値に収束するのですが、値が大きいと最適な重みを飛び越してしまう可能性もあります。逆に値が小さすぎると計算量が増えてしまうという点があります。

全てのパラメータに対してfor文でシンプルに計算をしています。

この重みの更新アルゴリズムは現状ではAdamというより改良されたものが一般的に利用されるようです。学習率を訓練が進むに連れて小さくしたりして、より最適な重みを効率よく見つけられるようになっているとのことでした。scikit-learnのMLPでもデフォルトでAdamが選択されます。

また、このようにシンプルな確率的勾配降下法出会っても、PyTorchにすでに実装されているものを利用した方がみやすいプログラムとすることができます。

In [ ]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

torch.optimを呼び出すことで、様々な最適化アルゴリズムを利用することができます。

#訓練
今まで個別にみてきた

・データをネットワークに入力

・勾配を求める

・バックプロパゲーションをする

という個別の動作を組み合わせて訓練の処理を作ることができます。

In [ ]:
#訓練ループ
optimizer.zero_grad()
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update